In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline

In [ ]:
results_file = 'benchmark-jdk-17.0.8-windows-i5-8400-throughput.txt'

result_pattern = re.compile(r"""
    BalancersBenchmark._(?P<threads>\d*)_threads?\s*
    (?P<balancer>\w*)Balancer\s*
    thrpt\s*
    (?P<count>\d*)\s*
    (?P<score>[\d]*),\d*\s*
    ±\s*
    (?P<error>[\d]*),\d*\s*
""", re.VERBOSE)

results = [];
with open(results_file, 'r', encoding='utf-8') as input:
    in_summary = False
    for line in input:
        in_summary = in_summary or 'Run complete' in line
        if in_summary:
            match = result_pattern.match(line)
            if match:
                results.append(match.groupdict())
            
data_long = pd.DataFrame.from_records(results)
data_long.head(5)

In [ ]:
data = pd.pivot_table(data_long, index='threads', columns='balancer', values='score')
#data.drop(['AtomicIntegerWeakCASPlain', 'AtomicIntegerWeakCASReleaseAcquire'], axis='columns', inplace=True)
data.index = data.index.astype(int)
data.sort_index(inplace=True)
colors = dict(zip(list(data.columns), list(mcolors.TABLEAU_COLORS)))
data

In [ ]:
data.drop(['Dummy1', 'Dummy2'], axis='columns').plot.bar(ylabel='ops/μs', color=colors)

In [ ]:
data.drop(['NonThreadSafe','Dummy1','Dummy2'], axis='columns').plot.bar(ylabel='ops/μs', color=colors)

In [ ]:
data[['Dummy1', 'Dummy2']].plot(ylabel='ops/μs', color=colors)

In [ ]:
data[['NonThreadSafe']].plot(ylabel='ops/μs', color=colors)

In [ ]:
data.drop(['NonThreadSafe', 'Dummy1', 'Dummy2'], axis='columns').plot(ylabel='ops/μs', color=colors)

In [ ]:
data.drop(1)[['NonThreadSafe']].plot(ylabel='ops/μs', color=colors)

In [ ]:
data.drop(1)[['Semaphore', 'SynchronizedBlock', 'SynchronizedMethod']].plot(ylabel='ops/μs', color=colors)

In [ ]:
data.drop(1)[['AtomicIntegerCAExchange', 'AtomicIntegerCASet', 'AtomicIntegerLambda']].plot(ylabel='ops/μs', color=colors)